In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns
import os
#import re
%matplotlib notebook

Convertion Equations

In [44]:
# Convert CH y/x to WGS lat
def CHtoWGSlat(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lat = (16.9023892 + (3.238272 * x_aux)) + \
            - (0.270978 * pow(y_aux, 2)) + \
            - (0.002528 * pow(x_aux, 2)) + \
            - (0.0447 * pow(y_aux, 2) * x_aux) + \
            - (0.0140 * pow(x_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lat = (lat * 100) / 36
    return lat

# Convert CH y/x to WGS long
def CHtoWGSlng(y, x):
    # Axiliary values (% Bern)
    y_aux = (y - 600000) / 1000000
    x_aux = (x - 200000) / 1000000
    lng = (2.6779094 + (4.728982 * y_aux) + \
            + (0.791484 * y_aux * x_aux) + \
            + (0.1306 * y_aux * pow(x_aux, 2))) + \
            - (0.0436 * pow(y_aux, 3))
    # Unit 10000" to 1" and convert seconds to degrees (dec)
    lng = (lng * 100) / 36
    return lng

Import Data

In [45]:
df_data = pd.read_csv("data.csv", header=None)

Wrangle Data

In [46]:
dw = 36 # Width of data
dh = 182 # Height of data
nc = len(df_data.columns) # Total number of columns
dr = int(nc/dw) - 3; # Number of data sets

df = pd.DataFrame()
df_x = df_data.loc[:, 0:(1*dw)-1]
df_y = df_data.loc[:, (1*dw):(2*dw)-1]
df_r = df_data.loc[:, (3*dw):(4*dw)-1]

In [47]:
x = []
x1 = []
x2 = []
x3 = []
x4 = []
x5 = []
x6 = []
x7 = []
x8 = []
y = []
y1 = []
y2 = []
y3 = []
y4 = []
y5 = []
y6 = []
y7 = []
y8 = []
r = []

for yy in range(1,len(df_x.columns)-1):
    for xx in range(1,len(df_x.index)-1):
        x.append(df_x.iloc[xx,yy])
        x1.append(df_x.iloc[xx-1,yy+1])
        x2.append(df_x.iloc[xx,yy+1])
        x3.append(df_x.iloc[xx+1,yy+1])
        x4.append(df_x.iloc[xx+1,yy])
        x5.append(df_x.iloc[xx+1,yy-1])
        x6.append(df_x.iloc[xx,yy-1])
        x7.append(df_x.iloc[xx-1,yy-1])
        x8.append(df_x.iloc[xx-1,yy])
        y.append(df_y.iloc[xx,yy])
        y1.append(df_y.iloc[xx-1,yy+1])
        y2.append(df_y.iloc[xx,yy+1])
        y3.append(df_y.iloc[xx+1,yy+1])
        y4.append(df_y.iloc[xx+1,yy])
        y5.append(df_y.iloc[xx+1,yy-1])
        y6.append(df_y.iloc[xx,yy-1])
        y7.append(df_y.iloc[xx-1,yy-1])
        y8.append(df_y.iloc[xx-1,yy])
        r.append(df_r.iloc[xx,yy])

df_j = pd.DataFrame(list(zip(x,x1,x2,x3,x4,x5,x6,x7,x8,y,y1,y2,y3,y4,y5,y6,y7,y8,r)),columns=['x','x1','x2','x3','x4','x5','x6','x7','x8','y','y1','y2','y3','y4','y5','y6','y7','y8','r'])

df_k = df_j[np.isfinite(df_j['r'])].reset_index(drop=True)
df_k = df_k[df_k['x'] != 0].reset_index(drop=True)
df_k = df_k[df_k['r'] != np.nan].reset_index(drop=True)
df_k["ntr_x"] = df_k[['x','x2', 'x3','x4']].mean(axis=1)
df_k["nbr_x"] = df_k[['x','x4', 'x5','x6']].mean(axis=1)
df_k["nbl_x"] = df_k[['x','x6', 'x7','x8']].mean(axis=1)
df_k["ntl_x"] = df_k[['x','x8', 'x1','x2']].mean(axis=1)
df_k["ntr_y"] = df_k[['y','y2', 'y3','y4']].mean(axis=1)
df_k["nbr_y"] = df_k[['y','y4', 'y5','y6']].mean(axis=1)
df_k["nbl_y"] = df_k[['y','y6', 'y7','y8']].mean(axis=1)
df_k["ntl_y"] = df_k[['y','y8', 'y1','y2']].mean(axis=1)

df_k["tr_x"] = df_k.apply(lambda x: CHtoWGSlat(x['ntr_x'], x['ntr_y']), axis=1)
df_k["tr_y"] = df_k.apply(lambda x: CHtoWGSlng(x['ntr_x'], x['ntr_y']), axis=1)
df_k["br_x"] = df_k.apply(lambda x: CHtoWGSlat(x['nbr_x'], x['nbr_y']), axis=1)
df_k["br_y"] = df_k.apply(lambda x: CHtoWGSlng(x['nbr_x'], x['nbr_y']), axis=1)
df_k["bl_x"] = df_k.apply(lambda x: CHtoWGSlat(x['nbl_x'], x['nbl_y']), axis=1)
df_k["bl_y"] = df_k.apply(lambda x: CHtoWGSlng(x['nbl_x'], x['nbl_y']), axis=1)
df_k["tl_x"] = df_k.apply(lambda x: CHtoWGSlat(x['ntl_x'], x['ntl_y']), axis=1)
df_k["tl_y"] = df_k.apply(lambda x: CHtoWGSlng(x['ntl_x'], x['ntl_y']), axis=1)

In [50]:
json = "["
for index, row in df_k.iterrows():
    json = json+'{"v":'+str(row['r'])+',"g":[['+str(row['tr_x'])+","+str(row['tr_y'])+"],["+str(row['br_x'])+","+str(row['br_y'])+"],["+str(row['bl_x'])+","+str(row['bl_y'])+"],["+str(row['tl_x'])+","+str(row['tl_y'])+"]]},"
json = json[:-1] + "]"

with open('data.json', 'w') as f:
    f.write(json)